In [3]:
import pandas as pd 
import numpy as np

In [4]:
data = pd.read_csv('hadoop.csv')

In [5]:
data = data.drop(['idx','access_date','access_time','access_page','access_referer','item_name'],axis=1)

In [6]:
unique_ip = data[data['item_type']=='cart']['access_ip'].unique()

## Make Dict_ip insert key is ip item is values

In [7]:
dict_ip = dict.fromkeys(unique_ip)   #dict key 생성

for i in dict_ip.keys():
    dict_ip[i] = []

#-----------------------------------------------#

for i in range(len(data)):          #dict value 삽입
    if (data['access_ip'].iloc[i] in dict_ip.keys()):
        key = data['access_ip'].iloc[i]
        dict_ip[key].append(data['item_id'].iloc[i])

In [8]:
for i,j in dict_ip.items():
    dict_ip[i] = list(map(str,j))        #dict value를 string으로 매핑
    
#------------------------------------------------#
#dict의 벨류가 4 미만이면 삭제
dict_ip = {key:value for key, value in dict_ip.items() if len(value) >4} 

In [9]:
cart_item = []       #dict의 마지막 아이템을 카트 아이템배열에 넣음
for i,j in dict_ip.items():
    cart_item.append(j[-1])

In [10]:
columns_name = []      #column이름을 string으로 변환
for i in range(14):
    columns_name.append(str(i))

## Make DataFrame New

In [11]:
df = pd.DataFrame.from_dict(dict_ip,orient='index',columns=columns_name)
df['13'] = cart_item

In [12]:
mask = df.applymap(lambda x: x is None)         #None값들 삭제
cols = df.columns[(mask).any()]
for col in df[cols]:
    df.loc[mask[col], col] = ''


In [13]:
item_list = []                 #item들을 뽑아서 한 row씩 한 리스트로 묶어 배열 생성
for i in range(len(df)):
    item_list.append(df.iloc[i,:].tolist())

# Word2Vec

In [14]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

In [15]:
path = get_tmpfile('word2vec.model')

In [21]:
model = Word2Vec(item_list, size=1, window=5, min_count=1,workers=4)
model.save('word2vec.model')

In [22]:
model = Word2Vec.load('word2vec.model')

In [23]:
word_vectors = model.wv  
vocabs = word_vectors.vocab.keys() #vocabs has keys that is item number

#word_vectors_list has values that is vector value from item number
word_vectors_list = [word_vectors[v] for v in vocabs] 

In [28]:
word_vectors_list

[array([0.53469443], dtype=float32),
 array([0.6020652], dtype=float32),
 array([0.6152077], dtype=float32),
 array([0.5917433], dtype=float32),
 array([0.60131764], dtype=float32),
 array([0.23415172], dtype=float32),
 array([0.30523935], dtype=float32),
 array([0.30623478], dtype=float32),
 array([0.29106972], dtype=float32),
 array([4.4761634], dtype=float32),
 array([1.7331383], dtype=float32),
 array([1.1064606], dtype=float32),
 array([1.824673], dtype=float32),
 array([2.0414894], dtype=float32),
 array([1.599892], dtype=float32),
 array([1.8786148], dtype=float32),
 array([1.8175789], dtype=float32),
 array([2.1381092], dtype=float32),
 array([1.5946205], dtype=float32),
 array([2.0484362], dtype=float32),
 array([1.5848309], dtype=float32),
 array([1.8368285], dtype=float32),
 array([1.616004], dtype=float32),
 array([1.5136415], dtype=float32),
 array([1.5802017], dtype=float32),
 array([1.3931156], dtype=float32),
 array([1.1917558], dtype=float32),
 array([3.9158576], dtype

In [18]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [19]:
# def plot_2d_graph(vocabs, xs, ys):
#     plt.figure(figsize=(12,6))
#     plt.scatter(xs,ys,marker='o')
#     for i, v in enumerate(vocabs):
#         plt.annotate(v,xy=(xs[i],ys[i]))

In [20]:
#pca = PCA(n_components=2)

In [21]:
# xys = pca.fit_transform(word_vectors_list)
# xs = xys[0]
# ys = xys[1]

In [22]:
# plot_2d_graph(vocabs,xs,ys)

In [23]:
for i in range(len(df)):            #word2vec value를 df로 넣음
    for j in range(len(df.iloc[i])):
        key=str(df.iloc[i,j])
        df.iloc[i,j] = word_vectors[key].tolist()[0]

In [24]:
df.index = range(len(df))

In [25]:
df = df.apply(pd.to_numeric)

# Data split && Learning

In [27]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [28]:
X_data = df.drop('13',axis=1)
y_data = df['13']

In [29]:
seed = 7
test_size = 0.3
X_train,X_test,y_train,y_test = train_test_split(X_data,y_data,test_size=test_size, random_state=seed)

In [ ]:
import 

# Test